# C(R)UD: toevoegen (insert), veranderen (update) en verwijderen (delete)

In het vorige notebook hebben aandacht besteed aan het opvragen van gegevens uit de datebase.
In dit notebook gaan we in op het toevoegen (insert of create), veranderen (update) en verwijderen (delete) van documenten in de database. We hebben dan de basisopdrachten (CRUD) van een database-systeem behandeld.

De onderstaande opdracht is nodig als initialisatie in sommige omgevingen.

In [1]:
import os
os.environ["PATH"]=os.environ["PATH"] + ":/usr/local/bin"

## Initialisaties

In [2]:
import os
import re
import pandas as pd
import numpy as np
from IPython.core.display import display, HTML
import pymongo
from bson.objectid import ObjectId

pd.set_option('max_colwidth',160)

userline = !echo $USER
username = userline[0]
dbname = username + "-demodb"
print("Database name: " + dbname)

print('Mongo version', pymongo.__version__)
client = pymongo.MongoClient('localhost', 27017)
db = client[dbname]
collection = db.contacts

collection.drop()
os.system('mongoimport -d ' + dbname + ' -c contacts adressen.json')

Database name: eelco-demodb
Mongo version 3.11.0


0

## Insert: toevoegen van documenten

Met de `insert`-opdracht kunnen we documenten aan een collectie toevoegen.
Vaak gebruiken we `insert_one`, om een enkel document toe te voegen.

In [3]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.insert_one(person)

In [4]:
list(collection.find({"name": "Sylvia Hansma"}))

[{'_id': ObjectId('5f96a4335359d765ea1d12ca'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 84', 'city': 'Halfweg'}}]

### Dubbele inserts

**Let op**: als je deze insert-opdracht herhaalt, voeg je nog  nog een kopie van hetzelfde document aan de collectie toe.
Dit is een nadeel van het laten genereren van de `_id` door MongoDB. (We zullen later behandelen hoe je documenten verwijdert.) 

**Opdracht** Ga dit na.

In [5]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 84", "city": "Halfweg"}
         }
collection.insert_one(person)
list(collection.find({"name": "Sylvia Hansma"}))

[{'_id': ObjectId('5f96a4335359d765ea1d12ca'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 84', 'city': 'Halfweg'}},
 {'_id': ObjectId('5f96a4365359d765ea1d12cb'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 84', 'city': 'Halfweg'}}]

## Update

Door middel van een update-opdracht kun je bepaalde velden veranderen of toevoegen (via `$set`, met een aantal nieuwe waarden).

**Veranderen** van gegevens in bestaande velden (properties):

In [6]:
upd_obj = {"address.street": "Mozartplein 73", "address.city": "Rotterdam", "address.postcode": "3021 BA"}

collection.update_one({"name": "Anna Verschuur"}, {"$set": upd_obj})
obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

{'_id': ObjectId('5f96a43237a74f1b5850d65c'), 'name': 'Anna Verschuur', 'email': 'anna@hotmail.com', 'address': {'street': 'Mozartplein 73', 'city': 'Rotterdam', 'postcode': '3021 BA'}}


**Opdracht** Maak een opdracht voor het aanpassen van het email-adres van Hans de Boer in de database.

In [7]:
upd_obj = {"email": "hansdb@ziggo.nl"}

collection.update_one({"name": "Hans de Boer"}, {"$set": upd_obj})
obj = collection.find_one({"name": "Hans de Boer"})
print(obj)

{'_id': ObjectId('5f96a43237a74f1b5850d658'), 'name': 'Hans de Boer', 'email': 'hansdb@ziggo.nl', 'tel': '06-1290 8746'}


**Toevoegen** van velden: als een veld in het `$set`-document niet bestaat, wordt dit toegevoegd. (Vgl. ook de *Upsert*, verderop.)

> Dit is in een relationele database alleen mogelijk als je de tabel verandert: je voegt dan één of meer velden voor alle rijen in de tabel toe. In MongoDB kun je dit *per document* doen.

In [8]:
collection.update_one({"name": "Anna Verschuur"}, {"$set": {"isFamily": True, "email": "anna33@gmail.com"}})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

{'_id': ObjectId('5f96a43237a74f1b5850d65c'), 'name': 'Anna Verschuur', 'email': 'anna33@gmail.com', 'address': {'street': 'Mozartplein 73', 'city': 'Rotterdam', 'postcode': '3021 BA'}, 'isFamily': True}


**Opdracht** Ga na dat velden op deze manier niet tweemaal toegevoegd worden.

In [9]:
collection.update_one({"name": "Anna Verschuur"}, {"$set": {"isFamily": True, "email": "anna33@gmail.com"}})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

{'_id': ObjectId('5f96a43237a74f1b5850d65c'), 'name': 'Anna Verschuur', 'email': 'anna33@gmail.com', 'address': {'street': 'Mozartplein 73', 'city': 'Rotterdam', 'postcode': '3021 BA'}, 'isFamily': True}


### Unset: verwijderen van velden (properties)

Je kunt een ongewenste veld (property) van een document verwijderen met de `$unset`-opdracht. De waarde van de property is in dit geval niet relevant.

> Deze `$unset` kan handig zijn als een foute update resulteert in het toevoegen van een nieuw veld in plaats van in een update van een bestaand veld.

In [10]:
upd_obj = {"isFamily": True}
collection.update_one({"name": "Anna Verschuur"}, {"$unset": upd_obj})

obj = collection.find_one({"name": "Anna Verschuur"})
print(obj)

{'_id': ObjectId('5f96a43237a74f1b5850d65c'), 'name': 'Anna Verschuur', 'email': 'anna33@gmail.com', 'address': {'street': 'Mozartplein 73', 'city': 'Rotterdam', 'postcode': '3021 BA'}}


## Upsert: overschrijven of toevoegen

Als je een document wilt toevoegen, waarbij je een eventueel bestaand document wilt overschrijven, dan kun je een update doen met `upsert=True`.
Dit betekent dat je een bestaand document overschrijft (update), of, als dat er niet is, een nieuw document toevoegt (insert).

> Hier moet je wel voorzichtig mee zijn: er kunnen bijvoorbeeld meerdere personen met dezelfde naam zijn. 
  Je kunt het query-deel van bijvoorbeeld uitbreiden met het email-adres. Met andere woorden: je moet een echte *key* gebruiken, waarmee je een document uniek identificeert. Je kunt natuurlijk ook de *primaire key* `_id` gebruiken.

In [11]:
person = {"name": "Sylvia Hansma", 
          "email": "sylh123@hotmail.com", 
          "address": {"street": "Rijksstraatweg 82", "city": "Halfweg"}
         }
collection.update_one({"name": person["name"]}, {"$set": person}, upsert=True)
list(collection.find())

[{'_id': ObjectId('5f96a43237a74f1b5850d657'),
  'name': 'Gijs Bennekom',
  'email': 'gijsbkom@ziggo.nl',
  'address': {'street': 'Lijsterlaan 132', 'city': 'Rotterdam'}},
 {'_id': ObjectId('5f96a43237a74f1b5850d658'),
  'name': 'Hans de Boer',
  'email': 'hansdb@ziggo.nl',
  'tel': '06-1290 8746'},
 {'_id': ObjectId('5f96a43237a74f1b5850d659'),
  'name': 'Joop de Zwart',
  'email': 'zwartejoop@ziggo.nl',
  'address': {'street': 'Rozengracht 42', 'city': 'Rotterdam'}},
 {'_id': ObjectId('5f96a43237a74f1b5850d65a'),
  'name': 'Leontien de Bruin',
  'email': ['lhmdebruin@hotmail.com', 'leontien134@tiscalimail.nl'],
  'address': {'street': 'Tulpstraat 17', 'city': 'Amsterdam'}},
 {'_id': ObjectId('5f96a43237a74f1b5850d65b'),
  'name': 'F.G. Schuitema',
  'address': {'street': 'Eikenlaan 23',
   'city': 'Amsterdam',
   'postcode': '1001 AB'}},
 {'_id': ObjectId('5f96a43237a74f1b5850d65c'),
  'name': 'Anna Verschuur',
  'email': 'anna33@gmail.com',
  'address': {'street': 'Mozartplein 73',


## Meervoudige waarden

Meervoudige waarden zijn wat lastiger aan te passen.
We geven een voorbeeld van het toevoegen van een email-adres voor een contactpersoon.
Het meeste werk hierbij gebeurt in Python.
We moeten er rekening mee houden dat (i) er nog geen email-veld is; (ii) het email-veld uit een enkele string bestaat; (iii) het nieuwe email-adres al in de lijst zit.

> Later zullen we zien hoe we het email-veld kunnen standaardiseren, wat deze update aanzienlijk vereenvoudigt.

**Opdracht** Ga na dat het niet uitmaakt of je deze opdracht éénmaal of vaker uitvoert: het resultaat blijft gelijk. (Het nieuwe mailadres wordt niet toegevoegd als het al aanwezig is.)

In [12]:
# add email "hdebo@gmail.com" to email-adresses of Hans de Boer

newemail = "hdebo@gmail.com"
obj = collection.find_one({"name": "Hans de Boer"})
print("before update:")
print(obj)
if "email" in obj:
    email = obj["email"]
    if type(email) == list:
        if newemail in email:
            pass # already in list
        else:
            email.append(newemail)
    elif email == newemail:
        email = [email]
    else:    
        email = [email, newemail]
else:
    email = [newemail]
upd_obj = {"email": email}
collection.update_one({"name": "Hans de Boer"}, {"$set": upd_obj})
obj = collection.find_one({"name": "Hans de Boer"})
print("after update")
print(obj)

before update:
{'_id': ObjectId('5f96a43237a74f1b5850d658'), 'name': 'Hans de Boer', 'email': 'hansdb@ziggo.nl', 'tel': '06-1290 8746'}
after update
{'_id': ObjectId('5f96a43237a74f1b5850d658'), 'name': 'Hans de Boer', 'email': ['hansdb@ziggo.nl', 'hdebo@gmail.com'], 'tel': '06-1290 8746'}


## Verwijderen van een document (delete)

Voor het verwijderen van documenten heb je twee soorten opdrachten:

* ``delete_one(filter-doc)``
* ``delete_many(filter-doc)``

De eerste opdracht verwijderd ten hoogste 1 document, ook als er meerdere documenten zijn die matchen met het filter-document.

Als er meerdere documenten zijn met dezelfde naam en andere eigenschappen, bijvoorbeeld doordat er per ongeluk een tweede `insert` uitgevoerd is, dan kun je het onbedoelde document verwijderen met behulp van de unieke `_id` in het filter

In [13]:
list(collection.find({"name": re.compile("Joop")}))

[{'_id': ObjectId('5f96a43237a74f1b5850d659'),
  'name': 'Joop de Zwart',
  'email': 'zwartejoop@ziggo.nl',
  'address': {'street': 'Rozengracht 42', 'city': 'Rotterdam'}}]

In [14]:
collection.delete_one({"name": "Joop de Zwart"})

list(collection.find({"name": re.compile("Joop")}))

[]

**Opdracht (a)** zorg ervoor dat er twee documenten zijn met dezelfde inhoud voor "Sylvia Hansma"`.

`(zie hierboven, dubbele inserts; waarschijnlijk heb je deze 2 documenten al.)`

**Opdracht (b)** Laat zien dat er twee (of meer) documenten zijn met dezelfde inhoud voor "Sylvia Hansma"`.

In [15]:
list(collection.find({"name": "Sylvia Hansma"}))

[{'_id': ObjectId('5f96a4335359d765ea1d12ca'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 82', 'city': 'Halfweg'}},
 {'_id': ObjectId('5f96a4365359d765ea1d12cb'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 84', 'city': 'Halfweg'}}]

**Opdracht (c)** Verwijder het onbedoelde document met behulp van de `_id` in het filter

`Opmerking: de ObjectId zal steeds verschillend zijn, dit is een voorbeeld.`

In [18]:
collection.delete_one({"_id":ObjectId('5f96a4365359d765ea1d12cb')})
list(collection.find({"name": "Sylvia Hansma"}))

[{'_id': ObjectId('5f96a4335359d765ea1d12ca'),
  'name': 'Sylvia Hansma',
  'email': 'sylh123@hotmail.com',
  'address': {'street': 'Rijksstraatweg 82', 'city': 'Halfweg'}}]

**Opdracht (d)** Formuleer een opdracht voor het verwijderen van alle documenten in `collection`. Laat zien dat de collection leeg is.

In [19]:
collection.delete_many({})

In [20]:
list(collection.find())

[]